# Training a convolutional neural network on neutrino interactions

- start by modifying `nn_inputs.py` with your desired parameters - this is so the testing notebook also has these available.

In [ ]:
from nn_functions import *
from nn_inputs import *

print("Number of NC images: ", len(pathNC), len(labelNC))
print("Number of CC images: ", len(pathCC), len(labelCC))
print(f"Training length is: {len(pathNC) - testsize_per_channel} per channel. Testing length is {testsize_per_channel} per channel.")

In [ ]:
# use CUDA if gpus are available 
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# choose criterion (loss function) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=1e-4)
print(f"Using {device}.")

Below is a preview of your image data:

In [ ]:
images,labels = next(iter(validation_loader))
fig, axis = plt.subplots(3, 5, figsize=(25, 20))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        npimg = images[i].numpy()
        npimg = np.transpose(npimg, (1, 2, 0))
        label = label_map[int(labels[i])]
        ax.imshow(npimg, cmap = "Greys_r")
        ax.set(title = f"{label}")

Train the CNN below:

In [ ]:
# train cnn 
TRAIN_LOSS = []
TRAIN_ACCURACY = []

for epoch in range(1, EPOCH_NUMBER + 1):
    epoch_loss = 0.0
    correct = 0
    total = 0
    for data_,target_ in train_loader:
        target_ = target_.to(device)
        data_ = data_.to(device)
        
        optimizer.zero_grad()
        # forward pass
        outputs = model(data_)
        loss = criterion(outputs,target_)
        
        # backwards pass
        loss.backward()
        optimizer.step()
        
        epoch_loss = epoch_loss + loss.item()
        _, pred = torch.max(outputs, dim = 1)
        correct = correct + torch.sum(pred == target_).item()
        total += target_.size(0)
    
    TRAIN_LOSS.append(epoch_loss)
    TRAIN_ACCURACY.append(100 * correct / total)
    print(f"Epoch {epoch}: Accuracy: {100 * correct/total}, Loss: {epoch_loss}")
    
    # plot confusion matrix during desired epochs
    if epoch % plot_frequency == 0:
        actual, predicted = eval_for_confmat(validation_loader, criterion = criterion)
        confmat = comp_confmat(actual, predicted)
        plot_confusion_matrix(confmat, f"{png_header}_{epoch}.png")
        # set model back to training mode 
        model.train()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(range(1, EPOCH_NUMBER + 1),TRAIN_LOSS)
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")

ax[1].plot(range(1, EPOCH_NUMBER + 1),TRAIN_ACCURACY)
ax[1].set_ylabel("Validation accuracy (%)")
ax[1].set_xlabel("Epoch")
plt.savefig(f"{png_header}_loss.png")
plt.show()

In [ ]:
# check initial seed and save model
torch.random.initial_seed()
torch.save(model, model_name)